# Description

The goal of this notebook is to create a basic pipeline to decontruct a piece and compose a new one using a network created from the original piece. This will use the instrucutions from Liu et. al to create a network from a single-line Telemann Flute Fantasie. 

Note: to use music21, get off anaconda:

    conda deactivate

In [1]:
import music21
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import queue

In [2]:
'''
Creat local corpus with access pieces
'''

localCorpus = music21.corpus.corpora.LocalCorpus()
localCorpus.addPath('../library')
music21.corpus.cacheMetadata()
#localCorpus.directoryPaths

/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: local metadata cache: starting processing of paths: 12
/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
bundles.py: WARNING: MetadataBundle Modification Time: 1587672929.497402
bundles.py: WARNING: Skipped 12 sources already in cache.
/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: writing time: 0.028 md items: 12

/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
caching.py: WARNING: cache: final writing time: 0.029 seconds


In [3]:
s = music21.corpus.parse('telemannfantasie1.xml')

In [4]:
s

<music21.stream.Score 0xa1a363250>

In [5]:
s.show("text")

{0.0} <music21.text.TextBox "IMSLP23680...">
{0.0} <music21.text.TextBox "2">
{0.0} <music21.text.TextBox "3">
{0.0} <music21.text.TextBox "4">
{0.0} <music21.metadata.Metadata object at 0xa1a430f10>
{0.0} <music21.stream.Part Flute>
    {0.0} <music21.instrument.Instrument 'P1: Flute: Flute (2)'>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.layout.PageLayout>
        {0.0} <music21.layout.SystemLayout>
        {0.0} <music21.layout.StaffLayout distance None, staffNumber 1, staffSize None, staffLines None>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.key.Key of A major>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Note A>
        {1.0} <music21.note.Note A>
        {1.25} <music21.note.Note B>
        {1.5} <music21.note.Note A>
        {1.75} <music21.note.Note E>
        {2.0} <music21.note.Note A>
        {2.25} <music21.note.Note C#>
        {2.5} <music21.note.Note A>
        {2.75} <music21.note.Note E>
  

In [6]:
flute = s[5]


In [7]:
for el in flute.recurse().notes:
    print(el.offset, el, el.activeSite)

0.0 <music21.note.Note A> <music21.stream.Measure 1 offset=0.0>
1.0 <music21.note.Note A> <music21.stream.Measure 1 offset=0.0>
1.25 <music21.note.Note B> <music21.stream.Measure 1 offset=0.0>
1.5 <music21.note.Note A> <music21.stream.Measure 1 offset=0.0>
1.75 <music21.note.Note E> <music21.stream.Measure 1 offset=0.0>
2.0 <music21.note.Note A> <music21.stream.Measure 1 offset=0.0>
2.25 <music21.note.Note C#> <music21.stream.Measure 1 offset=0.0>
2.5 <music21.note.Note A> <music21.stream.Measure 1 offset=0.0>
2.75 <music21.note.Note E> <music21.stream.Measure 1 offset=0.0>
3.0 <music21.note.Note A> <music21.stream.Measure 1 offset=0.0>
3.25 <music21.note.Note B> <music21.stream.Measure 1 offset=0.0>
3.5 <music21.note.Note A> <music21.stream.Measure 1 offset=0.0>
3.75 <music21.note.Note E> <music21.stream.Measure 1 offset=0.0>
0.0 <music21.note.Note A> <music21.stream.Measure 2 offset=4.0>
0.25 <music21.note.Note C#> <music21.stream.Measure 2 offset=4.0>
0.5 <music21.note.Note A> <musi

0.0 <music21.note.Note A> <music21.stream.Measure 72 offset=196.5>
0.25 <music21.note.Note G#> <music21.stream.Measure 72 offset=196.5>
0.5 <music21.note.Note A> <music21.stream.Measure 72 offset=196.5>
0.75 <music21.note.Note C#> <music21.stream.Measure 72 offset=196.5>
1.0 <music21.note.Note B> <music21.stream.Measure 72 offset=196.5>
1.25 <music21.note.Note A> <music21.stream.Measure 72 offset=196.5>
0.0 <music21.note.Note B> <music21.stream.Measure 73 offset=198.0>
0.5 <music21.note.Note E> <music21.stream.Measure 73 offset=198.0>
1.0 <music21.note.Note G#> <music21.stream.Measure 73 offset=198.0>
0.0 <music21.note.Note A> <music21.stream.Measure 74 offset=199.5>
0.75 <music21.note.Note C#> <music21.stream.Measure 74 offset=199.5>
1.0 <music21.note.Note B> <music21.stream.Measure 74 offset=199.5>
1.25 <music21.note.Note A> <music21.stream.Measure 74 offset=199.5>
0.0 <music21.note.Note D> <music21.stream.Measure 75 offset=201.0>
0.25 <music21.note.Note E> <music21.stream.Measure 75

In [8]:

def getMeasureFromNote(note_in_stream):
    return str(note_in_stream.activeSite).split()[1]

In [9]:
flute_notes =flute.recurse().notes
getMeasureFromNote(flute_notes[90])

'8'

# Formatting nodes and edges

In [10]:
notelst = list(flute_notes)
notelst

[<music21.note.Note A>,
 <music21.note.Note A>,
 <music21.note.Note B>,
 <music21.note.Note A>,
 <music21.note.Note E>,
 <music21.note.Note A>,
 <music21.note.Note C#>,
 <music21.note.Note A>,
 <music21.note.Note E>,
 <music21.note.Note A>,
 <music21.note.Note B>,
 <music21.note.Note A>,
 <music21.note.Note E>,
 <music21.note.Note A>,
 <music21.note.Note C#>,
 <music21.note.Note A>,
 <music21.note.Note E>,
 <music21.note.Note A>,
 <music21.note.Note E>,
 <music21.note.Note A>,
 <music21.note.Note E>,
 <music21.note.Note A>,
 <music21.note.Note E>,
 <music21.note.Note F#>,
 <music21.note.Note D>,
 <music21.note.Note E>,
 <music21.note.Note C#>,
 <music21.note.Note D>,
 <music21.note.Note B>,
 <music21.note.Note C#>,
 <music21.note.Note E>,
 <music21.note.Note F#>,
 <music21.note.Note D>,
 <music21.note.Note E>,
 <music21.note.Note C#>,
 <music21.note.Note D>,
 <music21.note.Note B>,
 <music21.note.Note C#>,
 <music21.note.Note A>,
 <music21.note.Note B>,
 <music21.note.Note C#>,
 <music

In [14]:
'''
Node Conversion Function (will be generic)
'''

def convert(lst):


    convert_note = lambda x: x.name+str(x.octave)
    nodelst = list(map(convert_note, lst))


    return nodelst

In [15]:
nodelst=convert(notelst)

In [16]:
len(nodelst)

686

# Create Graph

Use nodelst to create graph using networkx

In [19]:
def create_graph(nodelst):
    g = nx.MultiDiGraph()


    g.add_node(nodelst[0]) #adds first note
    i=1
    while i < len(nodelst):
        curNode = nodelst[i]
        #creates directed edge from previous note to current note   
        g.add_edge(nodelst[i-1], curNode) 
        i +=1
  
    #add start and end nodes
    g.add_edge("start", nodelst[0])

    if nodelst.count(nodelst[len(nodelst)-1]) != 1:
        g.add_node("end")
        g.add_edge(nodelst[len(nodelst)-1], "end")
    return g

    

In [ ]:
g=create_graph(nodelst)

In [ ]:
'''
Add start and end nodes
'''
nx.draw(g)

'''
pos=nx.spring_layout(g)
nx.draw_networkx_nodes(g,
                       pos,
                       nodelist=["start"],
                       node_color='g',
                       node_size=500,
                   alpha=0.8)

nx.draw_networkx_edges(g,
                       pos,
                       edgelist=[("start", nodelst[1])],
                       arrows=True)


nx.draw_networkx_nodes(g,
                       pos,
                       nodelist=["end"],
                       node_color='b',
                       node_size=500,
                   alpha=0.8)

nx.draw_networkx_edges(g,
                       pos,
                       edgelist=[(nodelst[len(nodelst)-2], "end")],
                       arrows=True)
                       
'''

In [ ]:
nx.write_gexf(g, "naive_composition.gexf")

# Random Walk

In [ ]:
def generate_tone(note_name):
        s = music21.stream.Stream()
        tsOneFour = music21.meter.TimeSignature('1/64')
        s.append(tsOneFour)
        print(note_name)
        n = music21.note.Note(note_name)
        n.duration.quarterLength=.01
        s.append(n)
        sp = music21.midi.realtime.StreamPlayer(s)
        sp.play()    

In [ ]:
s1 = music21.stream.Stream()
print('C4')
n = music21.note.Note('C4')
n.duration.quarterLength = .01
s1.append(n)
sp = music21.midi.realtime.StreamPlayer(s)
sp.play()    

In [ ]:
def show_node(node):
    pass

In [ ]:

def generate_randomwalk(g):
    get_neighbor = lambda x: x[1]
    randomwalk =[]
    cur_node = 'start'
    while cur_node != ['end']:
            edge_lst = list(g.edges(cur_node))
            neighbor_lst = list(map(get_neighbor, edge_lst))
            next_node = random.sample(neighbor_lst, 1)
            
            if cur_node[0] in nodelst: #not start or end
                '''
                record node
                '''
                randomwalk.append(cur_node[0])

        
        
            
            cur_node = next_node
    
    return randomwalk

    



In [ ]:
randomwalk=generate_randomwalk(g)


In [ ]:
randomwalk

# Convert to music

In [ ]:
def strto16thnote(randomwalk):
    notelist = []
    for string in randomwalk:
        n = music21.note.Note(string)
        n.duration.quarterLength = .25
        notelist.append(n)
    return notelist

In [ ]:
'''
Conversion function
'''
def convert_to_stream(randomwalk, conversion_function):
    notelist = conversion_function(randomwalk)
    s = music21.stream.Stream()
    for thisNote in notelist:
        s.append(thisNote)
    return s
        
    


In [ ]:
new_composition = convert_to_stream(randomwalk, strto16thnote)
sp = music21.midi.realtime.StreamPlayer(new_composition)
#sp.play()  

In [ ]:
new_composition.show()

In [ ]:
file = new_composition.write('midi', "new_composition1.mid")

# Manually deliminate sections

Manually delimit sections by choosing the bounds of a given encoding. Result: discreet sections. Weight of edge between section can be altered to change probability of length per section

In [ ]:
for el in s.recurse().notes:
    print(el.offset, el, el.activeSite)

In [11]:
'''
Delimit groupings at: m1, m5, m11, m27, m37, m49, m61, m75
Note: count starts at 1
'''
groups = [1, 5, 11, 27, 37, 49, 61, 75, "end"]



In [12]:
'''
Node Conversion Function (will be generic)
'''
def convert_grouping(lst, grouping):
    convert_note = lambda x: x.name+str(x.octave)

    nodelst=[] #list to store nodes
    #add first node
    transition_lst=[]
    i=0
    g=0
    node_group=grouping[g]
    while i < len(lst):
        note = lst[i]
        node_id = convert_note(note)
        #print(getMeasureFromNote(note))
        if getMeasureFromNote(note) == str(grouping[g]):
            node_group = grouping[g]
            g+=1
            if i !=0:
                transition_lst.append((nodelst[i-1],\
                     str(node_group)+" "+str(node_id)))
        node = str(node_group)+" "+str(node_id)
        nodelst.append(node)
        i +=1
    return nodelst, transition_lst


In [13]:
nodelst, t=convert_grouping(flute_notes, groups)

In [14]:
nodelst

['1 A4',
 '1 A4',
 '1 B4',
 '1 A4',
 '1 E4',
 '1 A4',
 '1 C#5',
 '1 A4',
 '1 E4',
 '1 A4',
 '1 B4',
 '1 A4',
 '1 E4',
 '1 A4',
 '1 C#5',
 '1 A4',
 '1 E4',
 '1 A4',
 '1 E5',
 '1 A4',
 '1 E4',
 '1 A4',
 '1 E5',
 '1 F#5',
 '1 D5',
 '1 E5',
 '1 C#5',
 '1 D5',
 '1 B4',
 '1 C#5',
 '1 E5',
 '1 F#5',
 '1 D5',
 '1 E5',
 '1 C#5',
 '1 D5',
 '1 B4',
 '1 C#5',
 '1 A4',
 '1 B4',
 '1 C#5',
 '1 D5',
 '1 E5',
 '1 F#5',
 '1 G#5',
 '1 A5',
 '1 E5',
 '1 C#5',
 '1 A4',
 '1 A5',
 '1 E5',
 '1 C#5',
 '1 A4',
 '1 A5',
 '1 A4',
 '5 D5',
 '5 C#5',
 '5 D5',
 '5 F#5',
 '5 D5',
 '5 A5',
 '5 D5',
 '5 C#5',
 '5 B4',
 '5 C#5',
 '5 E5',
 '5 C#5',
 '5 A5',
 '5 C#5',
 '5 D4',
 '5 C#5',
 '5 B4',
 '5 A4',
 '5 F#5',
 '5 C#5',
 '5 B4',
 '5 A4',
 '5 D4',
 '5 C#5',
 '5 B4',
 '5 A4',
 '5 F#5',
 '5 C#5',
 '5 B4',
 '5 A4',
 '5 D#4',
 '5 C#5',
 '5 B4',
 '5 A4',
 '5 A5',
 '5 C#5',
 '5 B4',
 '5 A4',
 '5 D#4',
 '5 C#5',
 '5 B4',
 '5 A4',
 '5 A5',
 '5 C#5',
 '5 B4',
 '5 A4',
 '5 E4',
 '5 G#4',
 '5 B4',
 '5 G#4',
 '5 E5',
 '5 B4',
 '5 

In [15]:
t

[('1 A4', '5 D5'),
 ('5 E4', '11 A4'),
 ('11 A4', '27 A5'),
 ('27 E4', '37 C#5'),
 ('37 E4', '49 C#5'),
 ('49 E4', '61 D5'),
 ('61 A4', '75 D5')]

## create graph

In [20]:
g=create_graph(nodelst)

In [21]:
g.edges

OutMultiEdgeView([('1 A4', '1 A4', 0), ('1 A4', '1 B4', 0), ('1 A4', '1 B4', 1), ('1 A4', '1 B4', 2), ('1 A4', '1 E4', 0), ('1 A4', '1 E4', 1), ('1 A4', '1 E4', 2), ('1 A4', '1 E4', 3), ('1 A4', '1 E4', 4), ('1 A4', '1 C#5', 0), ('1 A4', '1 C#5', 1), ('1 A4', '1 E5', 0), ('1 A4', '1 E5', 1), ('1 A4', '1 A5', 0), ('1 A4', '1 A5', 1), ('1 A4', '5 D5', 0), ('1 B4', '1 A4', 0), ('1 B4', '1 A4', 1), ('1 B4', '1 C#5', 0), ('1 B4', '1 C#5', 1), ('1 B4', '1 C#5', 2), ('1 E4', '1 A4', 0), ('1 E4', '1 A4', 1), ('1 E4', '1 A4', 2), ('1 E4', '1 A4', 3), ('1 E4', '1 A4', 4), ('1 C#5', '1 A4', 0), ('1 C#5', '1 A4', 1), ('1 C#5', '1 A4', 2), ('1 C#5', '1 A4', 3), ('1 C#5', '1 A4', 4), ('1 C#5', '1 D5', 0), ('1 C#5', '1 D5', 1), ('1 C#5', '1 D5', 2), ('1 C#5', '1 E5', 0), ('1 E5', '1 A4', 0), ('1 E5', '1 F#5', 0), ('1 E5', '1 F#5', 1), ('1 E5', '1 F#5', 2), ('1 E5', '1 C#5', 0), ('1 E5', '1 C#5', 1), ('1 E5', '1 C#5', 2), ('1 E5', '1 C#5', 3), ('1 F#5', '1 D5', 0), ('1 F#5', '1 D5', 1), ('1 F#5', '1 G

In [ ]:
nx.write_gexf(g, "grouped_composition.gexf")

## generate random walk

In [ ]:
randomwalk=generate_randomwalk(g)

## convert back function

In [ ]:
def group_strto16thnote(randomwalk):
    notelist = []
    i=0
    randomwalk.append('pad long')
    print(len(randomwalk))
    while i < len(randomwalk)-1:
        print(i)
        string = randomwalk[i]
        print(string)
        #note pitch
        notestr=string.split()[1]
        n = music21.note.Note(notestr)
        group_cur = string.split()[0]
        group_next = randomwalk[i+1].split()[0]
        #note duration
        if group_cur !=group_next:
            n.duration.quarterLength =2
        else:
            n.duration.quarterLength = .25 
        
        
        notelist.append(n)
        i +=1
    return notelist


In [ ]:
new_composition_group = convert_to_stream(randomwalk, group_strto16thnote)
sp = music21.midi.realtime.StreamPlayer(new_composition)

In [ ]:
new_composition_group.show()


In [ ]:
new_composition_group.write('midi', "new_composition_group1.mid")

# Degree Correction

In [22]:
'''
Node Conversion Function (will be generic)
'''

def convert_function(lst, grouping):
    convert_note = lambda x: x.name+str(x.octave)
    i=0
    g=0
    node_group=1
    nodelst=[]
    transition_nodes=[]
    while i < len(lst):
        note=lst[i]
        node_id = convert_note(note)
        #print(getMeasureFromNote(note))
        if getMeasureFromNote(note) == str(grouping[g]):
            node_group = grouping[g]
            g+=1
            if i !=0:
                prev_node=nodelst[i-1]
                transition_nodes.append( (prev_node, str(node_group)+" "+str(node_id)))
            
        node = str(node_group)+" "+str(node_id)
        nodelst.append(node)
        i +=1
    return nodelst, transition_nodes

In [23]:
nodelst, transition_nodes = convert_function(flute_notes, groups)

In [24]:
transition_nodes

[('1 A4', '5 D5'),
 ('5 E4', '11 A4'),
 ('11 A4', '27 A5'),
 ('27 E4', '37 C#5'),
 ('37 E4', '49 C#5'),
 ('49 E4', '61 D5'),
 ('61 A4', '75 D5')]

In [31]:
g1=create_graph(nodelst)

Strenghten transition nodes

In [26]:
# increase degree by 100

def degree_increase(graph, transition_lst, increase):
    for x in transition_lst:

        i=0
        while i<increase:
            graph.add_edge(x[0], x[1])
            i += 1
    return graph

In [27]:
g1 = degree_increase(g1, transition_nodes, 10)

In [28]:
g1.edges

OutMultiEdgeView([('1 A4', '1 A4', 0), ('1 A4', '1 B4', 0), ('1 A4', '1 B4', 1), ('1 A4', '1 B4', 2), ('1 A4', '1 E4', 0), ('1 A4', '1 E4', 1), ('1 A4', '1 E4', 2), ('1 A4', '1 E4', 3), ('1 A4', '1 E4', 4), ('1 A4', '1 C#5', 0), ('1 A4', '1 C#5', 1), ('1 A4', '1 E5', 0), ('1 A4', '1 E5', 1), ('1 A4', '1 A5', 0), ('1 A4', '1 A5', 1), ('1 A4', '5 D5', 0), ('1 A4', '5 D5', 1), ('1 A4', '5 D5', 2), ('1 A4', '5 D5', 3), ('1 A4', '5 D5', 4), ('1 A4', '5 D5', 5), ('1 A4', '5 D5', 6), ('1 A4', '5 D5', 7), ('1 A4', '5 D5', 8), ('1 A4', '5 D5', 9), ('1 A4', '5 D5', 10), ('1 B4', '1 A4', 0), ('1 B4', '1 A4', 1), ('1 B4', '1 C#5', 0), ('1 B4', '1 C#5', 1), ('1 B4', '1 C#5', 2), ('1 E4', '1 A4', 0), ('1 E4', '1 A4', 1), ('1 E4', '1 A4', 2), ('1 E4', '1 A4', 3), ('1 E4', '1 A4', 4), ('1 C#5', '1 A4', 0), ('1 C#5', '1 A4', 1), ('1 C#5', '1 A4', 2), ('1 C#5', '1 A4', 3), ('1 C#5', '1 A4', 4), ('1 C#5', '1 D5', 0), ('1 C#5', '1 D5', 1), ('1 C#5', '1 D5', 2), ('1 C#5', '1 E5', 0), ('1 E5', '1 A4', 0), (

In [29]:
nx.write_gexf(g1, "grouped_composition.gexf")

In [30]:
randomwalk_grouped=generate_randomwalk(g1)
new_composition_group = convert_to_stream(randomwalk_grouped, group_strto16thnote)
new_composition_group.write('midi', "new_composition_group2.mid")

NameError: name 'generate_randomwalk' is not defined

In [ ]:
nx.write_gexf(g, "grouped_composition_corrected.gexf")

In [ ]:
randomwalk1=generate_randomwalk(g)
group_length_corrected = convert_to_stream(randomwalk1, group_strto16thnote)
group_length_corrected.write('midi', "group_length_corrected.mid")

# Math-Decide on transition edge weight

Convert to graph with weighted edges

In [32]:
def convert_to_weighted(g):
    g_weight=nx.DiGraph()

    for edge in g.edges:
        num=g.number_of_edges(edge[0], edge[1])
        g_weight.add_edge(edge[0], edge[1], weight=num)

    for node in g_weight.nodes:
        neighbors = list(g_weight.neighbors(node))
        #print(neighbors)
        totalweight=0
        
        for n in neighbors:
            w=g_weight.get_edge_data(node, n)['weight'] #optimize by turning into dictionary, see documentation
            totalweight += w
        #print("new weights")
        for n in neighbors:
            w=g_weight.get_edge_data(node, n)['weight']
            g_weight.add_edge(node, n, weight=w/totalweight)
            #print(w/totalweight)
    return g_weight


In [33]:
g_weight = convert_to_weighted(g1)

In [34]:
g_weight.nodes()

NodeView(('1 A4', '1 B4', '1 E4', '1 C#5', '1 E5', '1 A5', '5 D5', '1 D5', '1 F#5', '1 G#5', '5 C#5', '5 F#5', '5 A5', '5 B4', '5 E5', '5 D4', '5 A4', '5 G#4', '5 G#5', '5 E4', '5 B5', '5 D#4', '11 A4', '11 D5', '11 C#5', '11 E5', '11 A5', '11 G#4', '11 B4', '11 G4', '11 D4', '11 C#6', '11 G5', '27 A5', '11 F#4', '11 F#5', '11 E4', '11 B5', '11 A#4', '11 G#5', '27 E5', '27 C#5', '27 F#5', '27 G#4', '27 F#4', '27 E4', '27 D#4', '27 B5', '27 D5', '27 A4', '27 B4', '27 D#5', '27 G5', '37 C#5', '37 D5', '37 B4', '37 F#5', '37 E5', '37 E4', '37 D#5', '37 F#4', '37 G#4', '37 A4', '49 C#5', '37 G#5', '37 A5', '49 D5', '49 B4', '49 F#5', '49 E5', '49 E4', '49 D#5', '49 F#4', '49 G#4', '49 A4', '61 D5', '49 G#5', '49 A5', '61 E5', '61 C#5', '61 B4', '61 A5', '61 F#5', '61 A4', '61 E4', '75 D5', '61 G#5', '61 G#4', '75 E5', '75 C#5', '75 B4', '75 A5', '75 F#5', '75 A4', '75 E4', 'end', '75 G#5', '75 G#4', 'start'))

calculate probability of taking path on length shortest, shortest+1, shortest+2.....

In [35]:
def gen_path_histogram(subgraph1,  startnode,  outgoing_edge, additional_depth=5, outgoing_edge_weight=5):
    #set outgoing edge weight
    subgraph = subgraph1.copy()

    subgraph=degree_increase(subgraph,\
                            [ outgoing_edge], outgoing_edge_weight)

    subgraph = convert_to_weighted(subgraph)
    
  
    endnode=outgoing_edge[1]
    
    '''
    Breath-first search
    '''


    q = queue.Queue(maxsize=0)
    q.put([1, startnode, 1]) #length, topnode, weight
    

    
    shortest_path_length = nx.shortest_path_length(subgraph, \
                            source=startnode, target=endnode)
    
    total_searched_length = additional_depth+shortest_path_length+1
    #stores probability of each path length
    finished = np.zeros(total_searched_length) 
    
    while 1:
        curpath = q.get()
        length=curpath[0]
        curnode=curpath[1]
        weight=curpath[2]
        print("current path length is ", length)
        
        #stopping condition
        if length == total_searched_length: 
            break

        
        #path found
        if curnode == endnode:
            finished[length]+=weight
            
        #create lengthened qpath and add to queue
        else:
            for n in list(subgraph.neighbors(curnode)):
                         
                l=length+1
                w = weight *  subgraph.get_edge_data(curnode, n)['weight']
                q.put([l, n, w])
        
          
    return finished

    
            
            
            
            
    
    
    
    
    
    
    
    
    
    
    

In [36]:
t

[('1 A4', '5 D5'),
 ('5 E4', '11 A4'),
 ('11 A4', '27 A5'),
 ('27 E4', '37 C#5'),
 ('37 E4', '49 C#5'),
 ('49 E4', '61 D5'),
 ('61 A4', '75 D5')]

In [37]:
subgraph = g.subgraph(['1 A4', \
        '1 B4', '1 E4', '1 C#5', '1 E5', '1 A5', \
                              '1 D5', '1 F#5', '1 G#5', '5 D5'])

In [38]:
subgraph.edges()

OutMultiEdgeDataView([('1 C#5', '1 A4'), ('1 C#5', '1 A4'), ('1 C#5', '1 A4'), ('1 C#5', '1 A4'), ('1 C#5', '1 A4'), ('1 C#5', '1 D5'), ('1 C#5', '1 D5'), ('1 C#5', '1 D5'), ('1 C#5', '1 E5'), ('1 D5', '1 E5'), ('1 D5', '1 E5'), ('1 D5', '1 E5'), ('1 D5', '1 B4'), ('1 D5', '1 B4'), ('1 E5', '1 A4'), ('1 E5', '1 F#5'), ('1 E5', '1 F#5'), ('1 E5', '1 F#5'), ('1 E5', '1 C#5'), ('1 E5', '1 C#5'), ('1 E5', '1 C#5'), ('1 E5', '1 C#5'), ('1 A5', '1 E5'), ('1 A5', '1 E5'), ('1 A5', '1 A4'), ('1 A4', '1 A4'), ('1 A4', '1 B4'), ('1 A4', '1 B4'), ('1 A4', '1 B4'), ('1 A4', '1 E4'), ('1 A4', '1 E4'), ('1 A4', '1 E4'), ('1 A4', '1 E4'), ('1 A4', '1 E4'), ('1 A4', '1 C#5'), ('1 A4', '1 C#5'), ('1 A4', '1 E5'), ('1 A4', '1 E5'), ('1 A4', '1 A5'), ('1 A4', '1 A5'), ('1 A4', '5 D5'), ('1 E4', '1 A4'), ('1 E4', '1 A4'), ('1 E4', '1 A4'), ('1 E4', '1 A4'), ('1 E4', '1 A4'), ('1 B4', '1 A4'), ('1 B4', '1 A4'), ('1 B4', '1 C#5'), ('1 B4', '1 C#5'), ('1 B4', '1 C#5'), ('1 G#5', '1 A5'), ('1 F#5', '1 D5'), (

In [39]:
arr = gen_path_histogram(subgraph,  '1 A4',  ('1 A4', '5 D5'), outgoing_edge_weight=1, additional_depth=3)



current path length is  1
current path length is  2
current path length is  2
current path length is  2
current path length is  2
current path length is  2
current path length is  2
current path length is  2
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  3
current path length is  4
current path length is  4
current path length is  4
current path length is  4
current path length is  4
current path length is  4
current path length is  4
current path length is  4
current path length is  4
current path length is  4
current path length is  4
current path length is  4
current path

In [40]:
arr

array([0.        , 0.        , 0.11764706, 0.00692042, 0.05734672])

In [30]:
arr

array([0.        , 0.        , 0.28571429, 0.01360544, 0.11259043,
       0.03450157, 0.05542905])

In [ ]:
x=list(range(3,10))

In [ ]:
x

# Pyvis visuals

In [ ]:
net = Network()
net.from_nx(g)

In [ ]:
gr = nx.Graph()

In [ ]:
gr.add_node('a', section=1)

In [ ]:
gr.add_edge('a', 'b')

In [ ]:
gr.add_node('a', section=2)

In [ ]:
gr.nodes

In [ ]:
class Qpath:
        def __init__(self,length, nodelist, weight):
            self.length=length
            self.nodelist=nodelist
            self.weight=weight
            

In [ ]:
k = Qpath(1,[2,3], 4)

In [ ]:
k.length

In [ ]:
stack = [0, 1, 2]
stack.append(3)
stack

In [ ]:
stack.pop()

In [ ]:
stack

In [ ]:
def gen_path_histogram(subgraph1,  startnode,  outgoing_edge, additional_depth=5, outgoing_edge_weight=5):
    #set outgoing edge weight
    subgraph = subgraph1.copy()

    subgraph=degree_increase(subgraph,\
                            [ outgoing_edge], outgoing_edge_weight)

    subgraph = convert_to_weighted(subgraph)
    
  
    endnode=outgoing_edge[1]
    
    '''
    Breath-first search
    '''


    
    
    
    def worker():
        while True:
            curpath = q.get()
            length=curpath[0]
            curnode=curpath[1]
            weight=curpath[2]
            print("current path length is ", length)

            #stopping condition
            if length == total_searched_length: 
                break

            if curpath is None:
                print("curpath is None")
                break
            

            #path found
            if curnode == endnode:
                finished[length]+=weight

            #create lengthened qpath and add to queue
            else:
                for n in list(subgraph.neighbors(curnode)):

                    length=length+1
                    weight = weight *  subgraph.get_edge_data(curnode, n)['weight']
                    q.put([length, n, weight])
                    
            q.task_done()


             
    q = queue.Queue()
    q.put([1, startnode, 1]) #length, topnode, weight  
    
    shortest_path_length = nx.shortest_path_length(subgraph, \
                            source=startnode, target=endnode)
    
    total_searched_length = additional_depth+shortest_path_length+1
    #stores probability of each path length
    finished = np.zeros(total_searched_length) 
    num_worker_threads=4
    
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)

    for item in source():
        q.put(item)

    # block until all tasks are done
    q.join()

    return finished

    
            
            
            
            
    
    
    
    
    
    
    
    
    
    
    